In [4]:
import numpy as np
import argparse
import json
import sys
import os
import math
from scipy.optimize import fsolve
from scipy import log
import random
import datetime


# External libraries
import torch
from torch import optim

# Internal libraries

import models
import posteriors
import priors
import hawkes_model, excitation_kernels
import learners
import utils

from make_data_for_samples import make_data              #多个样本数据
from make_data_for_estimate import make_estimate_data    #单个样本数据

def make_object(module, name, args):
    return getattr(module, name)(**args)

#def learn_vi(events, end_time, vi_seed, adjacency_true, inference_param_dict, return_learner=False):
def learn_vi(events, vi_seed, inference_param_dict, return_learner=False):
    # Extract some parameters for easier access
    #n_nodes = len(events)
    n_events = len(events)
    n_nodes = len(events[0])
    M = inference_param_dict['excitation']['args'].get('M', 1)
    n_params = n_nodes * (n_nodes * M + 1)
    n_edges = M * n_nodes ** 2
    # Set seed
    np.random.seed(vi_seed)
    # Set starting pointM * n_nodes ** 2
    x0 = torch.tensor(
        np.hstack((
            np.hstack((  # alpha, the mean of the parameters
                np.random.normal(loc=0.1, scale=0.1, size=n_nodes),
                np.random.normal(loc=0.1, scale=0.1, size=n_edges),)),
            np.hstack((  # beta=log(sigma), log of the variance of the parameters
                np.log(np.clip(np.random.normal(loc=0.2, scale=0.1, size=n_nodes), 1e-1, 2.0)),
                np.log(np.clip(np.random.normal(loc=0.2, scale=0.1, size=n_edges), 1e-1, 2.0)),))
        )),
        dtype=torch.float64, requires_grad=True
    )
    # Init Hawkes process model object
    excitation_obj = make_object(excitation_kernels, **inference_param_dict['excitation'])
    hawkes_model_obj = hawkes_model.HawkesModel(excitation=excitation_obj, verbose=False)
    # Init the posterior object
    posterior_obj = make_object(posteriors, **inference_param_dict['posterior'])
    # Init the prior object
    prior_type = inference_param_dict['prior']['name']
    prior_args = inference_param_dict['prior']['args']
    prior_args['C'] = torch.tensor(prior_args['C'], dtype=torch.float64)  # cast to tensor
    prior_obj = make_object(priors, prior_type, prior_args)
    # Init the variational inference model object
    model = models.ModelHawkesVariational(
        model=hawkes_model_obj, posterior=posterior_obj, prior=prior_obj,
        **inference_param_dict['model']['args'])
   
    # Init the optimizer
    opt_type = inference_param_dict['optimizer']['name']
    opt_args = inference_param_dict['optimizer']['args']
    opt = getattr(optim, opt_type)([x0], **opt_args)
    # Init learner
    learner = learners.VariationalInferenceLearner(
        model=model, optimizer=opt, **inference_param_dict['learner']['args'])
    # Fit the model
    events_t = [torch.tensor(events_i) for events_i in events]  # cast to tensor
    #learner.fit(events_t, end_time, x0, callback=callback)
    learner.fit(events_t, x0=x0, callback=None)
    print()
    if return_learner:
        return learner
    # Extract the mode of the posterior
    z_est_mode = learner.model.posterior.mode(learner.coeffs[:n_params], learner.coeffs[n_params:])
    adj_est_ora = z_est_mode[n_nodes:].detach()
    mu_est_ora = z_est_mode[:n_nodes].detach()
    adj_est_ora = adj_est_ora.view(n_nodes, n_nodes, M)
    adj_est = z_est_mode[n_nodes:].detach().numpy()
    adj_est = np.reshape(adj_est, (n_nodes, n_nodes, M)).sum(-1).ravel()
    mu_est = z_est_mode[:n_nodes].detach().numpy()
    #mu_est = np.reshape(mu_est,n_nodes).ravel()
    coeffs_est = learner.coeffs.detach().numpy()
    #log_like_sum,min_intens,log_like,end_time = hawkes_model_obj.log_likelihood(mu_est_ora,adj_est_ora)
    log_like_sum,intens_sum,integral_instesity,end_time = hawkes_model_obj.log_likelihood(mu_est_ora,adj_est_ora)
    
    return coeffs_est, adj_est,mu_est,intens_sum,integral_instesity,end_time


def run(exp_dir, param_filename, output_filename, stdout=None, stderr=None):
    # Reset random seed
    np.random.seed(None)

    if stdout is not None:
        sys.stdout = open(stdout, 'w')
    if stderr is not None:
        sys.stderr = open(stderr, 'w')

    print('\nExperiment parameters')
    print('=====================')
    print(f'        exp_dir = {exp_dir:s}')
    print(f' param_filename = {param_filename:s}')
    print(f'output_filename = {output_filename:s}')
    print(flush=True)
    print('\nStart time is: ', datetime.datetime.today())

    result_dict = {}
    
    data_fileName = "./data/DSL-StrongPasswordData.xls"
    global events
    events = make_data('s032',10400,10410,data_fileName)
    n_jumps_per_dim = list(map(len, events[0]))
    print('\nNumber of jumps:', len(events)*sum(n_jumps_per_dim))
    print('\nper node:', n_jumps_per_dim)
    
    C_list = [1.0]*132
    
    param_dict={'inference':{'vi_exp':{'excitation': {'name': 'ExponentialKernel','args': {'decay': 0.3, 'cut_off': 1000.0}}, 
                          'posterior': {'name': 'LogNormalPosterior', 'args': {}},
                          'prior': {'name': 'GaussianLaplacianPrior', 'args': {'dim': 11, 'n_params': 132, 'C': C_list}}, 
                          'model': {'args': {'n_samples': 1, 'n_weights': 1, 'weight_temp': 1.0}}, 
                          'optimizer': {'name': 'Adam', 'args': {'lr': 0.01}}, 
                          'learner': {'args': {'tol': 1e-04, 'lr_gamma': 0.9999, 'max_iter': 40000, 'hyperparam_momentum': 0.5, 'hyperparam_interval': 100, 'hyperparam_offset': 0}}}}
               }
    

    print('\nINFERENCE')
    print('=========')

    for key, inference_param_dict in param_dict['inference'].items():
        if key.startswith('vi'):
            print(f'\nRun VI ({key:s})')
            print('------')
            # Set random seed (for reproducibility)
            np.random.seed()  # Reset random number generator to avoid dependency on simulation seed
            #vi_seed = np.random.randint(2**32 - 1)
            vi_seed = np.random.randint(2**16 - 1)
            print(f'vi random seed: {vi_seed}')
            # Run inference
            global intens_sum
            global integral_instesity
            #coeffs_var, adj_var, mu_var, nu, varsigma = learn_vi(events, vi_seed, inference_param_dict)
            coeffs_var, adj_var, mu_var,intens_sum,integral_instesity,end_time  = learn_vi(events, vi_seed, inference_param_dict)
            #模型参数
            adj_var = adj_var.ravel()
            mu_var = mu_var.ravel()            
          
            global  end_time_result
            end_time_result = [0.0]*len(end_time)
            
            for i in range(len(end_time)):
                end_time_result[i]=end_time[i].tolist()
                
            expresion_temp = ''
            events_n = len(events)
            dim = len(events[0])
            
            for i in range(events_n):
                temp = ''                  
                for j in range(dim):
                    temp += 'log('+str(intens_sum[i][j].detach().numpy())+ '-'+ 'epsilon_noise['+str(i)+'])'+ '+'   
                #print(intens)
                expresion_temp += temp[:-1] + '-' + str(integral_instesity[i].detach().numpy()) + '+' + str(dim) +'*'+ str(end_time_result[i])+'*'+ 'epsilon_noise['+str(i)+'],'

            expresion = expresion_temp[:-1]
            expresion = '['+expresion+']'
         
            result_dict.update({
                key: {
                    'vi_seed': vi_seed,             # VI random seed
                    'coeffs': coeffs_var.tolist(),  # VI parameters
                    'adjacency': adj_var.tolist(),  # VI Estimator
                    'mu':  mu_var.tolist(),
                    'expresion': expresion,
                }
            })
 

    print('\n\nSave results...')
    
    print('\ncoeffs:',  coeffs_var.tolist())
    print( '\nadjacency:', adj_var.tolist())
    print('\nmu:', mu_var.tolist())
    #print('\nnu:',nu)
    #print('\nvarsigma:',varsigma)

    with open(os.path.join(exp_dir, output_filename), 'w') as output_file:
        json.dump(result_dict, output_file)

    # Log that the run is finished
    print('\n\nFinished.')
    print('\nEnd time is: ', datetime.datetime.today())


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-d', '--dir', dest='dir', type=str,
                        #required=True, help="Working directory")
                        required=False, default=".")
    parser.add_argument('-p', '--params', dest='param_filename', type=str,
                        required=False, default='params.json',
                        help="Input parameter file (JSON)")
    parser.add_argument('-o', '--outfile', dest='output_filename', type=str,
                        required=False, default='output.json',
                        help="Output file (JSON)")
    #args = parser.parse_args()
    args = parser.parse_known_args()[0]

    #run(exp_dir=args.dir, param_filename=args.param_filename,output_filename=args.output_filename)
    run ('.','params.json','penalty10_decay0.3+s032_10400-10410.json')


Experiment parameters
        exp_dir = .
 param_filename = params.json
output_filename = penalty10_decay0.3+s032_10400-10410.json


Start time is:  2020-03-17 13:17:47.603268

Number of jumps: 220

per node: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

INFERENCE

Run VI (vi_exp)
------
vi random seed: 41543
end_iter is: 39276
Converged!



Save results...

coeffs: [-0.48712535939632673, -0.6291089668478922, -0.8163064690369258, -0.9501784033879538, -0.9111588394791271, -0.9744105341481439, -2.150387413093768, -2.4785814327781517, -2.6476130298174274, -2.711587023303444, -2.644317224080956, -3.4208108579366856, -3.6166799520027197, -3.6199357282475586, -3.5671878948688955, -3.4979559672257077, -3.3901077672749733, -3.328659447827607, -3.26422530457173, -3.2481468142864593, -3.1503409624372387, -2.9159048300256494, -2.1086613447205624, -3.391859979009077, -3.628428993499887, -3.577536623986946, -3.507222230420168, -3.3941832189940775, -3.3373590947030776, -3.2885733070378844, -3.231531313032937,